<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/conditional_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN (cGAN)

---
## 目的
条件付きのGAN (conditional GAN; cGAN) によって意図した画像の生成をして動作を理解する．
## モジュールのインポート

はじめに必要となるモジュールをインポートします．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算を行うことが可能です．
Falseとなっている場合は，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

In [ ]:
import os
import zipfile
import urllib.request
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## ネットワークの構築

cGANのネットワーク構造を定義します．


基本的なネットワーク構造は，通常のGANと同様に全結合層で設計します．
これに加えて，cGANでは条件の入力があるため，Generator及びDiscriminatorの入力層のチャネル数をクラス数分多く定義します．

<img src="https://dl.dropboxusercontent.com/s/42tfm9viymbqemo/cGAN.png" width=50%>



In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=100, n_cls=10):
        super(Generator, self).__init__()
        self.model = nn.Sequential()

        self.model.add_module('fc1', nn.Linear(latent_dim + n_cls, 128))
        self.model.add_module('act1', nn.ReLU(inplace=True))
        self.model.add_module('fc2', nn.Linear(128, 32 * 32))
        self.model.add_module('act2', nn.ReLU(inplace=True))
        self.weight_init()

    def weight_init(self):
        for m in self.model:
            if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
                m.bias.data.zero_()

    def forward(self, z):
        return self.model(z)
    
class Discriminator(nn.Module):
    def __init__(self, n_input=32*32, n_cls=10, img_size=32):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential()
        
        self.model.add_module('fc1', nn.Linear(n_input+n_cls, 128))
        self.model.add_module('act1', nn.ReLU(inplace=True))        
        self.model.add_module('fc2', nn.Linear(128, 1))
        self.weight_init()
  
    def weight_init(self):
        for m in self.model:
            if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
                m.bias.data.zero_()

    def forward(self, x):
        return self.model(x).view(-1)

## データセットと最適化関数
データセットにはMNISTを使用します．
最適化関数はAdam optimizer使用し，学習率$2\times 10^4$，betaの値を$0.5, 0.999$として学習します．

In [ ]:
transform_training = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform_training, download=True)
training_data = DataLoader(mnist_data, batch_size=100, shuffle=True)


latent_dim = 100
cls_num = 10
G = Generator(latent_dim=latent_dim, n_cls=cls_num)
D = Discriminator(n_input=32*32, n_cls=cls_num)
if use_cuda:
    G = G.cuda()
    D = D.cuda()

opt_g = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_d = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

## GANの学習

cGANの学習をします．

cGANでは以下の誤差関数で学習を行います．

$$\min _{G} \max _{D} V(D, G)=\mathbb{E}_{x \sim p_{\text {data }}({x})}[\log D(x \mid y)]+\mathbb{E}_{{z} \sim p_{z}({z})}[\log (1-D(G({z} \mid {y})))]$$

ここで，$x$は実画像，$y$は条件を示しています．$G(z|y)$は条件$y$を与えたときに生成した画像です．

このように，条件を与えて生成した画像を判定することで，意図した画像を生成することを可能としています．


In [ ]:
n_epoch = 20
n_critic = 1
criterion = nn.BCEWithLogitsLoss()
if use_cuda:
    criterion = criterion.cuda()

for epoch in range(1, n_epoch+1):
    Tensor = torch.cuda.FloatTensor
    for idx, (real_x, y) in enumerate(training_data):
        if use_cuda:
            real_x = real_x.cuda()
            
        batch = real_x.size(0)
        real_x = real_x.view(batch, -1)
        onehot = torch.eye(cls_num)[y]
        if use_cuda:
            onehot = onehot.cuda()
        onehot_expand = onehot.expand(batch, onehot.size(1))
        flag_real = Tensor(batch).fill_(1.0)
        flag_fake = Tensor(batch).fill_(0.0)
        
        for _ in range(n_critic):
            D.zero_grad()
            z = torch.randn(batch, latent_dim+cls_num)
            if use_cuda:
                z = z.cuda()
            z[:, :10] = onehot
            fake_x = G(z)
            _real_x = torch.cat((real_x, onehot_expand), dim=1)
            _fake_x = torch.cat((fake_x, onehot_expand), dim=1)
            out_real = D(_real_x)
            out_fake = D(_fake_x.detach())
            loss_real = criterion(out_real, flag_real)
            loss_fake = criterion(out_fake, flag_fake)
            dis_loss = loss_real + loss_fake
            dis_loss.backward()
            opt_d.step()
            
        G.zero_grad()
        z = torch.randn(batch, latent_dim+cls_num)
        if use_cuda:
            z = z.cuda()
        z[:, :10] = onehot
        fake_x = G(z)
        fake_x = torch.cat((fake_x, onehot_expand), dim=1)
        out_gen = D(fake_x)
        gen_loss = criterion(out_gen, flag_real)
        gen_loss.backward()
        opt_g.step()
        
        if idx % 100 == 0:
            print('Training epoch: {} [{}/{} ({:.0f}%)] | D loss: {:.6f} | G loss: {:.6f} |'\
                  .format(epoch, idx * len(real_x), len(training_data.dataset),
                  100. * idx / len(training_data), dis_loss.item(), gen_loss.item()))

## 生成画像の確認
学習済みのGeneratorを用いてどのような画像が生成されるのか，狙った画像を生成することができるのかを確認しましょう．

In [ ]:
c = np.arange(10, dtype=np.int32)
onehot = torch.eye(10)[c].cuda()[:,:]
z = torch.randn(len(c), 100+10)
if use_cuda:
    z = z.cuda()
z[:, :10] = onehot

test_img = G(z)

if use_cuda:
    test_img = test_img.cpu()
_test_img = (test_img * 256.).clamp(min=0., max=255.).data.view(len(c), 32, 32).numpy()

fig = plt.figure(figsize=(10, 3))
for i, im in enumerate(_test_img):
    ax = fig.add_subplot(3, 10, i+1, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

# 課題

1. 条件を固定して線形補間したとき，どのような変化が現れるか確認してください．生成画像AからBへの線形補間は，[0, 1]で線形増加する値$\alpha$を用いて以下の式で表せます．
$$
\hat{z} = (1-\alpha)\cdot z_{A} + \alpha\cdot z_{B}\\
I_{A\rightarrow B} = G(\hat{z})
$$

# 参考文献
[1] Mehdi Mirza and Simon Osindero, Conditional Generative Adversarial Nets, arXiv, 2014.\
[2] Alec Radford, Luke Metz and Soumith Chintala, Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks, ICLR, 2016.